In [2]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
import bs4
from bs4 import BeautifulSoup
 

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hemanthbodala/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
#! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz

## Read Data

In [4]:
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz'
data = pd.read_table(url,sep = '\t', error_bad_lines=False) 

/Users/hemanthbodala/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 16148: expected 15 fields, saw 22\nSkipping line 20100: expected 15 fields, saw 22\nSkipping line 45178: expected 15 fields, saw 22\nSkipping line 48700: expected 15 fields, saw 22\nSkipping line 63331: expected 15 fields, saw 22\n'
b'Skipping line 86053: expected 15 fields, saw 22\nSkipping line 88858: expected 15 fields, saw 22\nSkipping line 115017: expected 15 fields, saw 22\n'
b'Skipping line 137366: expected 15 fields, saw 22\nSkipping line 139110: expected 15 fields, saw 22\nSkipping line 165540: expected 15 fields, saw 22\nSkipping line 171813: expected 15 fields, saw 22\n'
b'Skipping line 203723: expected 15 fields, saw 22\nSkipping line 209366: expected 15 fields, saw 22\nSkipping line 211310: expected 1

b'Skipping line 1968846: expected 15 fields, saw 22\nSkipping line 1999941: expected 15 fields, saw 22\nSkipping line 2001492: expected 15 fields, saw 22\nSkipping line 2011204: expected 15 fields, saw 22\nSkipping line 2025295: expected 15 fields, saw 22\n'
b'Skipping line 2041266: expected 15 fields, saw 22\nSkipping line 2073314: expected 15 fields, saw 22\nSkipping line 2080133: expected 15 fields, saw 22\nSkipping line 2088521: expected 15 fields, saw 22\n'
b'Skipping line 2103490: expected 15 fields, saw 22\nSkipping line 2115278: expected 15 fields, saw 22\nSkipping line 2153174: expected 15 fields, saw 22\nSkipping line 2161731: expected 15 fields, saw 22\n'
b'Skipping line 2165250: expected 15 fields, saw 22\nSkipping line 2175132: expected 15 fields, saw 22\nSkipping line 2206817: expected 15 fields, saw 22\nSkipping line 2215848: expected 15 fields, saw 22\nSkipping line 2223811: expected 15 fields, saw 22\n'
b'Skipping line 2257265: expected 15 fields, saw 22\nSkipping line

In [5]:
print((data['star_rating']==1).sum(),",",(data['star_rating']==2).sum(),",",(data['star_rating']==3).sum(),",",(data['star_rating']==4).sum(),",",(data['star_rating']==5).sum())
data.head(3)

426900 , 241948 , 349547 , 731733 , 3124759


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,37000337,R3DT59XH7HXR9K,B00303FI0G,529320574,Arthur Court Paper Towel Holder,Kitchen,5.0,0.0,0.0,N,Y,Beautiful. Looks great on counter,Beautiful. Looks great on counter.,2015-08-31
1,US,15272914,R1LFS11BNASSU8,B00JCZKZN6,274237558,Olde Thompson Bavaria Glass Salt and Pepper Mi...,Kitchen,5.0,0.0,1.0,N,Y,Awesome & Self-ness,I personally have 5 days sets and have also bo...,2015-08-31
2,US,36137863,R296RT05AG0AF6,B00JLIKA5C,544675303,Progressive International PL8 Professional Man...,Kitchen,5.0,0.0,0.0,N,Y,Fabulous and worth every penny,Fabulous and worth every penny. Used for clean...,2015-08-31


## Keep Reviews and Ratings

In [6]:
data1 = data.drop(['marketplace', 'customer_id', 'product_id', 'product_category', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_date', 'review_id', 'product_parent', 'product_title'], axis=1)

In [7]:
data1=data1.dropna()
data1 = data1.reset_index(drop=True)

In [8]:
nval1 = data1.isna().sum()
print(len(nval1))

2


In [52]:
len(data1.index)
data1.head(3)

,star_rating,review_body
0,5.0,Beautiful. Looks great on counter.
1,5.0,I personally have 5 days sets and have also bo...
2,5.0,Fabulous and worth every penny. Used for clean...


# Labelling Reviews:
## The reviews with rating 4,5 are labelled to be 1 and 1,2 are labelled as 0. Discard the reviews with rating 3'

In [53]:
data2 = data1[data1.star_rating != 3]
data2.loc[data2.star_rating < 3, "label"] = 0
data2.loc[data2.star_rating > 3, "label"] = 1
print((data1['star_rating']==3).sum(),",",(data2['label']==0).sum(),",",(data2['label']==1).sum())

data2.head(3)

349539 , 668809 , 3856296


/Users/hemanthbodala/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/hemanthbodala/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,star_rating,review_body,label
0,5.0,Beautiful. Looks great on counter.,1.0
1,5.0,I personally have 5 days sets and have also bo...,1.0
2,5.0,Fabulous and worth every penny. Used for clean...,1.0


 ## We select 200000 reviews randomly with 100,000 positive and 100,000 negative reviews.



In [54]:
data_p = data2
data_p = data2[data2.label != 1]
data_n = data2

data_n = data2[data2.label != 0]
data_p.head()
len(data_n.index)
data_pf = data_p.sample(n = 100000, random_state=1)
len(data_pf.index)
data_pf.head()
data_nf = data_n.sample(n = 100000, random_state=1)
len(data_nf.index)
data_nf.head()
frames = [data_pf, data_nf]
data_final = pd.concat(frames)
result_len3 = data_final['review_body'].map(str).apply(len).mean()
data_final.head(3)

,star_rating,review_body,label
2060619,1.0,its too small,0.0
2431470,2.0,The burr grinder creates uniform ground coffee...,0.0
2063439,2.0,My muffins were sticking to my non-stick muffi...,0.0


# Data Cleaning

## Convert the all reviews into the lower case.

In [55]:
data_final['review_body'] = data_final['review_body'].str.lower()
data_final.head(3)

,star_rating,review_body,label
2060619,1.0,its too small,0.0
2431470,2.0,the burr grinder creates uniform ground coffee...,0.0
2063439,2.0,my muffins were sticking to my non-stick muffi...,0.0


## remove the HTML and URLs from the reviews

In [56]:
import re
import bs4
from bs4 import BeautifulSoup
data_final['review_body']=data_final['review_body'].apply(lambda x: re.sub(r"http\S+", "", x))
#data_final['review_body'] = data_final.review_body.apply(lambda x: re.split('https:\/\/.*', str(x))[0])
data_final['review_body']=data_final['review_body'].apply(lambda x: BeautifulSoup(x).get_text())
#data_final['review_body'] = data_final.review_body.apply(lambda x: bs4.BeautifulSoup(x, 'lxml').get_text())

/Users/hemanthbodala/opt/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:332: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [57]:
data_final.head(3)

,star_rating,review_body,label
2060619,1.0,its too small,0.0
2431470,2.0,the burr grinder creates uniform ground coffee...,0.0
2063439,2.0,my muffins were sticking to my non-stick muffi...,0.0


## remove non-alphabetical characters

In [58]:
data_final.head()
import contractions
def contractionfunction(s):
    s = contractions.fix(s)

    return s
data_final['review_body']=data_final['review_body'].apply(lambda x:contractionfunction(x))
#data_final.review_body.str.replace('[^a-z]+', ' ')
data_final['review_body']=data_final['review_body'].apply(lambda x: " ".join([re.sub('[^a-z]+','', x) for x in nltk.word_tokenize(x)]))

In [59]:
data_final.head(3)

,star_rating,review_body,label
2060619,1.0,its too small,0.0
2431470,2.0,the burr grinder creates uniform ground coffee...,0.0
2063439,2.0,my muffins were sticking to my nonstick muffin...,0.0


## Remove the extra spaces between the words

In [60]:
#data_r1.str.replace('  ', '')
data_final['review_body'] = data_final.review_body.replace('\s+', ' ', regex=True)
#data_r3.str.count("                                                            ").sum()
data_final.head(3)

,star_rating,review_body,label
2060619,1.0,its too small,0.0
2431470,2.0,the burr grinder creates uniform ground coffee...,0.0
2063439,2.0,my muffins were sticking to my nonstick muffin...,0.0


## perform contractions on the reviews.

In [61]:
#import contractions
#def contractionfunction(s):
#    s = contractions.fix(s)

#    return s
#data_final['review_body']=data_final['review_body'].apply(lambda x:contractionfunction(x))
result_len4 = data_final['review_body'].map(str).apply(len).mean()
print(result_len3,",",result_len4)
data_final.head(3)

323.274835 , 309.18604


,star_rating,review_body,label
2060619,1.0,its too small,0.0
2431470,2.0,the burr grinder creates uniform ground coffee...,0.0
2063439,2.0,my muffins were sticking to my nonstick muffin...,0.0


# Pre-processing

## remove the stop words 

In [62]:
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
data_final['review_body']=data_final['review_body'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))
data_final.head(3)

,star_rating,review_body,label
2060619,1.0,small,0.0
2431470,2.0,burr grinder creates uniform ground coffeeit e...,0.0
2063439,2.0,muffins sticking nonstick muffin pan tried usi...,0.0


## perform lemmatization  

In [63]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
data_final['review_body']=data_final['review_body'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))
data_final.head(3)

,star_rating,review_body,label
2060619,1.0,small,0.0
2431470,2.0,burr grinder creates uniform ground coffeeit e...,0.0
2063439,2.0,muffin sticking nonstick muffin pan tried usin...,0.0


In [64]:
result_len2 = data_final['review_body'].apply(len).mean()
print(result_len4,",",result_len2)
data_final.head(3)

309.18604 , 189.56203


,star_rating,review_body,label
2060619,1.0,small,0.0
2431470,2.0,burr grinder creates uniform ground coffeeit e...,0.0
2063439,2.0,muffin sticking nonstick muffin pan tried usin...,0.0


# TF-IDF Feature Extraction

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(data_final['review_body'],data_final['label'], test_size=0.20, random_state=30)
print("Train:",X_train.shape,Y_train.shape,"Test: ",(X_test.shape,Y_test.shape))

vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

Train: (160000,) (160000,) Test:  ((40000,), (40000,))


# Perceptron

In [66]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [67]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
ppm = Perceptron(max_iter=50, eta0=0.1,random_state=0)
ppm.fit(tf_x_train,Y_train)
y_test_pred = ppm.predict(tf_x_test)
y_train_pred = ppm.predict(tf_x_train)

In [73]:
from sklearn.metrics import classification_report
report_train=classification_report(Y_train, y_train_pred,output_dict=True)
#print("Perceptron Model:")
#print("Report for Traning data Classification")
#print("Accuracy:",report_train['accuracy'])
#print("Precision:",precision_score(Y_train, y_train_pred))
#print("Recall:",recall_score(Y_train, y_train_pred))
#print("f1_score:",f1_score(Y_train, y_train_pred))
#print(report_train)
print(report_train['accuracy'],",",precision_score(Y_train, y_train_pred),",",recall_score(Y_train, y_train_pred),",",f1_score(Y_train, y_train_pred))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
report_test=classification_report(Y_test, y_test_pred,output_dict=True)
#print("Report for Testing data Classification")
#print("Accuracy:",report_test['accuracy'])
#print("Precision:",precision_score(Y_test, y_test_pred))
#print("Recall:",recall_score(Y_test, y_test_pred))
#print("f1_score:",f1_score(Y_test, y_test_pred))
#print(report_test)
print(report_test['accuracy'],",",precision_score(Y_test, y_test_pred),",",recall_score(Y_test, y_test_pred),",",f1_score(Y_test, y_test_pred))

0.888925 , 0.8979179197112985 , 0.8775027826065207 , 0.8875929767747812
0.868575 , 0.881884881678206 , 0.851739108737961 , 0.866549894651334


# SVM

In [74]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0)
clf.fit(tf_x_train,Y_train)
y_test_pred=clf.predict(tf_x_test)
y_train_pred=clf.predict(tf_x_train)
from sklearn.metrics import classification_report
report_train=classification_report(Y_train, y_train_pred,output_dict=True)

print(report_train['accuracy'],",",precision_score(Y_train, y_train_pred),",",recall_score(Y_train, y_train_pred),",",f1_score(Y_train, y_train_pred))
report=classification_report(Y_test, y_test_pred,output_dict=True)

print(report['accuracy'],",",precision_score(Y_test, y_test_pred),",",recall_score(Y_test, y_test_pred),",",f1_score(Y_test, y_test_pred))

0.93696875 , 0.9381270064205457 , 0.9355810957841948 , 0.9368523214677061
0.89365 , 0.8967475996581712 , 0.8902140825390489 , 0.8934688971251127


# Logistic Regression

In [75]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000)
clf.fit(tf_x_train,Y_train)
y_test_pred=clf.predict(tf_x_test)
y_train_pred=clf.predict(tf_x_train)
from sklearn.metrics import classification_report
report_train=classification_report(Y_train, y_train_pred,output_dict=True)
#print("Logistic Regression Model:")
#print("Report for Traning data Classification")
#print("Accuracy:",report_train['accuracy'])
#print("Precision:",precision_score(Y_train, y_train_pred))
#print("Recall:",recall_score(Y_train, y_train_pred))
#print("f1_score:",f1_score(Y_train, y_train_pred))
#print(report_train)
print(report_train['accuracy'],",",precision_score(Y_train, y_train_pred),",",recall_score(Y_train, y_train_pred),",",f1_score(Y_train, y_train_pred))
report=classification_report(Y_test, y_test_pred,output_dict=True)
#print("Report for Testing data Classification")
#print("Accuracy:",report['accuracy'])
#print("Precision:",precision_score(Y_test, y_test_pred))
#print("Recall:",recall_score(Y_test, y_test_pred))
#print("f1_score:",f1_score(Y_test, y_test_pred))
#print(report)
print(report['accuracy'],",",precision_score(Y_test, y_test_pred),",",recall_score(Y_test, y_test_pred),",",f1_score(Y_test, y_test_pred))

0.91255625 , 0.9159835548717431 , 0.9083428171233476 , 0.9121471853317006
0.895075 , 0.8995661824051655 , 0.889914666400519 , 0.894714396809071


# Naive Bayes

In [76]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(tf_x_train,Y_train)
y_test_pred=classifier.predict(tf_x_test)
y_train_pred=classifier.predict(tf_x_train)
report_train=classification_report(Y_train, y_train_pred,output_dict=True)
#print("Naive Bayes Model:")
#print("Report for Traning data Classification")
#print("Accuracy:",report_train['accuracy'])
#print("Precision:",precision_score(Y_train, y_train_pred))
#print("Recall:",recall_score(Y_train, y_train_pred))
#print("f1_score:",f1_score(Y_train, y_train_pred))
#print(report_train)
print(report_train['accuracy'],",",precision_score(Y_train, y_train_pred),",",recall_score(Y_train, y_train_pred),",",f1_score(Y_train, y_train_pred))
report=classification_report(Y_test, y_test_pred,output_dict=True)
#print("Report for Testing data Classification")
#print("Accuracy:",report['accuracy'])
#print("Precision:",precision_score(Y_test, y_test_pred))
#print("Recall:",recall_score(Y_test, y_test_pred))
#print("f1_score:",f1_score(Y_test, y_test_pred))
#print(report)
print(report['accuracy'],",",precision_score(Y_test, y_test_pred),",",recall_score(Y_test, y_test_pred),",",f1_score(Y_test, y_test_pred))

0.888925 , 0.8979179197112985 , 0.8775027826065207 , 0.8875929767747812
0.868575 , 0.881884881678206 , 0.851739108737961 , 0.866549894651334


In [82]:
import nltk
import sklearn
print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The nltk version is 3.5.
The scikit-learn version is 0.23.2.
